In [2]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

## Downloading LLAVA Instruct 150 k dataset 

In [3]:
! wget https://huggingface.co/datasets/liuhaotian/LLaVA-Instruct-150K/resolve/main/llava_instruct_150k.json

--2024-02-12 10:51:32--  https://huggingface.co/datasets/liuhaotian/LLaVA-Instruct-150K/resolve/main/llava_instruct_150k.json
Resolving huggingface.co (huggingface.co)... 18.239.50.16, 18.239.50.80, 18.239.50.103, ...
Connecting to huggingface.co (huggingface.co)|18.239.50.16|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/4d/41/4d41ea1e2709f0e68e9e361e4218192b9620c5a3f2cb8055bc625942b6cd3039/6b68bc5ca2bfd8a71119af0e8454929668ccda6a334955ccc95d114fc8d082fa?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27llava_instruct_150k.json%3B+filename%3D%22llava_instruct_150k.json%22%3B&response-content-type=application%2Fjson&Expires=1707994292&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwNzk5NDI5Mn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy80ZC80MS80ZDQxZWExZTI3MDlmMGU2OGU5ZTM2MWU0MjE4MTkyYjk2MjBjNWEzZjJjYjgwNTViYzYyNTk0MmI2Y2QzMDM5LzZiNjhiY

In [5]:
import json
# Opening JSON file - instruct150k
f = open('llava_instruct_150k.json')

# returns JSON object as
# a dictionary
data = json.load(f)
data[0]

{'id': '000000033471',
 'image': '000000033471.jpg',
 'conversations': [{'from': 'human',
   'value': '<image>\nWhat are the colors of the bus in the image?'},
  {'from': 'gpt', 'value': 'The bus in the image is white and red.'},
  {'from': 'human',
   'value': 'What feature can be seen on the back of the bus?'},
  {'from': 'gpt', 'value': 'The back of the bus features an advertisement.'},
  {'from': 'human',
   'value': 'Is the bus driving down the street or pulled off to the side?'},
  {'from': 'gpt',
   'value': 'The bus is driving down the street, which is crowded with people and other vehicles.'}]}

## Preparing data 

In [9]:
# Data Preparation
data_instruct150_flatten = []  # Initialize an empty list to store flattened data
r = 0  # Counter for printing progress

# Iterate over each item in the 'data' list
for a_idx, d in enumerate(data):
    # Extract image information
    image = d['image']
    image_url = 'http://images.cocodataset.org/train2017/' + image
    
    # Iterate over conversations
    conv_iter = iter(d['conversations'])
    for i in conv_iter:
        # Get next conversation item (assumed to be GPT response)
        gpt_ans = next(conv_iter)
        
        # Filter out long answers
        if len(gpt_ans['value']) > 200:
            continue
        
        # Add to flattened list if it's a human-to-GPT conversation
        if i['from'] == 'human' and gpt_ans['from'] == 'gpt':
            # Append image URL, human message, and GPT response
            data_instruct150_flatten.append((image_url, i['value'].replace('<image>\n', '').replace('\n<image>', ''), gpt_ans['value']))
    
    # Print progress every 10,000 items
    if a_idx % 10000 == 0:
        print(f"{10000 * r} processed")
        r += 1


0 processed
10000 processed
20000 processed
30000 processed
40000 processed
50000 processed
60000 processed
70000 processed
80000 processed
90000 processed
100000 processed
110000 processed
120000 processed
130000 processed
140000 processed
150000 processed


In [10]:
data_instruct150_flatten[4]

('http://images.cocodataset.org/train2017/000000052846.jpg',
 'What is the cat doing in the image?',
 'The cat is coming out from some curtains onto the couch and is sitting or standing on top of it.')

In [11]:
# add tokens
phi_model_name  = "microsoft/phi-2"
tokenizer  = AutoTokenizer.from_pretrained(phi_model_name, trust_remote_code=True)
tokenizer.add_tokens('[QA]')
tokenizer.add_special_tokens({'pad_token':'[PAD]'}) 
tokenizer.pad_token, tokenizer.eos_token

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


('[PAD]', '<|endoftext|>')

In [13]:
import csv
# gpt like training dataset
with open('train_token.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows([['img_url','input','label']])
    
train_data_temp = []
r = 1
for df in data_instruct150_flatten:
  image_url = df[0]
  image_q   = df[1] + ' [QA]'
  image_a   = df[2] +  tokenizer.eos_token
  #print(image_q)
  #print(image_a)
  
  # tokenise 
  ques_token = tokenizer(image_q, return_tensors="pt", return_attention_mask=False)['input_ids'].squeeze(0)
  ans_token  = tokenizer(image_a, return_tensors="pt", return_attention_mask=False)['input_ids'].squeeze(0)

  #print(ques_token)
  #print(ans_token)
  #break

  context_length = len(ques_token)
  combo_q_a = torch.cat( [ques_token,ans_token])
    
  for al in range(len(ans_token)):   
    input = combo_q_a[al : al + context_length].numpy()
    label = combo_q_a[al + 1 : al + context_length + 1].numpy()
    train_data_temp.append([image_url,input,label])
    if len(train_data_temp) >= 100000: # write to the file
       print(f"Writing to disk after {r * 100000} rows")
       r += 1
       with open('train_token.csv', 'a', newline='') as file:
          writer = csv.writer(file)
          writer.writerows(train_data_temp)
       train_data_temp = []

Writing to disk after 100000 rows
Writing to disk after 200000 rows
Writing to disk after 300000 rows
Writing to disk after 400000 rows
Writing to disk after 500000 rows
Writing to disk after 600000 rows
Writing to disk after 700000 rows
Writing to disk after 800000 rows
Writing to disk after 900000 rows
Writing to disk after 1000000 rows
Writing to disk after 1100000 rows
Writing to disk after 1200000 rows
Writing to disk after 1300000 rows
Writing to disk after 1400000 rows
Writing to disk after 1500000 rows
Writing to disk after 1600000 rows
Writing to disk after 1700000 rows
Writing to disk after 1800000 rows
Writing to disk after 1900000 rows
Writing to disk after 2000000 rows
Writing to disk after 2100000 rows
Writing to disk after 2200000 rows
Writing to disk after 2300000 rows
Writing to disk after 2400000 rows
Writing to disk after 2500000 rows
Writing to disk after 2600000 rows
Writing to disk after 2700000 rows
Writing to disk after 2800000 rows
Writing to disk after 2900000

In [14]:
train_data_temp[0]

['http://images.cocodataset.org/train2017/000000431378.jpg',
 array([19916,   448,    11,   810,   339,   318,  9361,   284,  5529,
          465,  5236,   290]),
 array([ 448,   11,  810,  339,  318, 9361,  284, 5529,  465, 5236,  290,
        6594])]

In [15]:
import torch
from torch.utils.data import Dataset
from PIL import Image
from torch.utils.data import random_split, DataLoader
from transformers import AutoProcessor, AutoTokenizer
import pickle

## Training Data:

In [16]:
df_data = pd.read_csv('train_token.csv')
df_data.head(10)

,img_url,input,label
0,http://images.cocodataset.org/train2017/000000...,[ 2061 389 262 7577 286 262 1323 2...,[ 389 262 7577 286 262 1323 287 2...
1,http://images.cocodataset.org/train2017/000000...,[ 389 262 7577 286 262 1323 287 2...,[ 262 7577 286 262 1323 287 262 29...
2,http://images.cocodataset.org/train2017/000000...,[ 262 7577 286 262 1323 287 262 29...,[ 7577 286 262 1323 287 262 2939 ...
3,http://images.cocodataset.org/train2017/000000...,[ 7577 286 262 1323 287 262 2939 ...,[ 286 262 1323 287 262 2939 30 2...
4,http://images.cocodataset.org/train2017/000000...,[ 286 262 1323 287 262 2939 30 2...,[ 262 1323 287 262 2939 30 220 502...
5,http://images.cocodataset.org/train2017/000000...,[ 262 1323 287 262 2939 30 220 502...,[ 1323 287 262 2939 30 220 50295 4...
6,http://images.cocodataset.org/train2017/000000...,[ 1323 287 262 2939 30 220 50295 4...,[ 287 262 2939 30 220 50295 464 13...
7,http://images.cocodataset.org/train2017/000000...,[ 287 262 2939 30 220 50295 464 13...,[ 262 2939 30 220 50295 464 1323 2...
8,http://images.cocodataset.org/train2017/000000...,[ 262 2939 30 220 50295 464 1323 2...,[ 2939 30 220 50295 464 1323 287 2...
9,http://images.cocodataset.org/train2017/000000...,[ 2939 30 220 50295 464 1323 287 2...,[ 30 220 50295 464 1323 287 262 29...


In [17]:
for i in (df_data[0:1]['input']):
    print(i)
for i in (df_data[0:1]['label']):
    print(i)

[ 2061   389   262  7577   286   262  1323   287   262  2939    30   220
 50295]
[  389   262  7577   286   262  1323   287   262  2939    30   220 50295
   464]


In [19]:
class llavadataset(Dataset):
  def __init__(self, qa_dataset, phi_model_name, clip_model_name, tokenizer):
    self.processor  = AutoProcessor.from_pretrained(clip_model_name)
    self.qa_dataset = qa_dataset

  def __len__(self):
    return self.qa_dataset.shape[0]

  def __getitem__(self, idx):
    # from image perspective
    img_url = self.qa_dataset.img_url[idx]
    ques    = torch.tensor(np.array(np.matrix(self.qa_dataset.input[idx]))[0])  
    ans     = torch.tensor(np.array(np.matrix(self.qa_dataset.label[idx]))[0])
    
    # image load
    image_load = Image.open(requests.get(img_url,stream=True).raw)
    image_processed = self.processor(images=image_load, return_tensors="pt") ['pixel_values']
    image_processed = image_processed.squeeze(0)
    # q = self.tokenizer(ques, return_tensors="pt", return_attention_mask=False)['input_ids'].squeeze(0)
    # a = self.tokenizer(ans, return_tensors="pt", return_attention_mask=False)['input_ids'].squeeze(0)
    return(image_processed , ques, ans)

In [20]:
clip_model_name = "wkcn/TinyCLIP-ViT-61M-32-Text-29M-LAION400M"
#phi_model_name  = "microsoft/phi-2"
#tokenizer  = AutoTokenizer.from_pretrained(phi_model_name, trust_remote_code=True)
csv_file = 'train_token.csv'
qa_dataset = pd.read_csv(csv_file)
step2_dataset = llavadataset(qa_dataset, phi_model_name, clip_model_name, tokenizer)

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.60k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

In [22]:
def collate_fn(batch):
    image_embeddings, ques, ans = zip(*batch)
    image_embeddings_stacked = torch.stack(image_embeddings, dim=0)
    ques_padded = torch.nn.utils.rnn.pad_sequence(ques, batch_first=True, padding_value=tokenizer.pad_token_id)
    ans_padded = torch.nn.utils.rnn.pad_sequence(ans, batch_first=True, padding_value=tokenizer.pad_token_id)
    return (image_embeddings_stacked, ques_padded,ans_padded)

In [23]:
tokenizer.pad_token_id,tokenizer.eos_token_id

(50296, 50256)

In [30]:
# Get the token value for token ID 50295
token_value = tokenizer.convert_ids_to_tokens(50295)
token_value

'[QA]'

## Validation data loader

In [32]:
val_dataloader   = DataLoader(llavadataset(qa_dataset, phi_model_name,clip_model_name,tokenizer),
                      collate_fn=collate_fn, batch_size=2, num_workers = 10, shuffle=True, pin_memory=True)

/opt/conda/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [33]:
import random

In [34]:
#! wget https://huggingface.co/datasets/liuhaotian/LLaVA-Instruct-150K/resolve/main/llava_instruct_150k.json
# Opening JSON file - instruct150k
f = open('llava_instruct_150k.json')

# returns JSON object as
# a dictionary
data = json.load(f)

In [35]:
# create input pickle file by flattening the data
data_instruct150_sample_val_flatten = []
r = 0

for a_idx,d in enumerate(data):
    image = d['image']
    image_url = 'http://images.cocodataset.org/train2017/' + image
    conv_iter = iter( d['conversations'])
    for i in conv_iter:
      gpt_ans = next(conv_iter)
      if len(gpt_ans['value']) > 200: # filter out too long answers
          continue
      if i['from'] == 'human' and gpt_ans['from'] == 'gpt':
        image_q   = i['value'].replace('<image>\n','').replace('\n<image>','') + ' [QA]'
        image_a   = gpt_ans['value'] +  tokenizer.eos_token
        data_instruct150_sample_val_flatten.append([image_url, image_q, image_a ])

    if a_idx % 10000 == 0:
      print(f"{10000 * r} processed")
      r += 1
      if r >= 2:
        break

0 processed
10000 processed


In [36]:
data_instruct150_sample_val_flatten[3]

['http://images.cocodataset.org/train2017/000000052846.jpg',
 'Where is the cat positioned in the image? [QA]',
 'The cat is positioned on top of the back of the couch in the living room.<|endoftext|>']

In [37]:
# header
with open('sample_val_data.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows([['img_url','q','a']])

# data
with open('sample_val_data.csv', 'a', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(data_instruct150_sample_val_flatten)